In [6]:
import pandas as pd
import numpy as np


In [7]:
csv_path = "csv/movies.csv"
movies = pd.read_csv(csv_path, encoding = "ISO-8859-1")
#movies.head(3)

FileNotFoundError: [Errno 2] File b'csv/movies.csv' does not exist: b'csv/movies.csv'

In [3]:
movies = movies.rename(columns={"name": "film"})
#movies.head(3)

# Oscar Csv

In [4]:
csv_path = "csv/the_oscar_award.csv"
oscars = pd.read_csv(csv_path, encoding = "ISO-8859-1")
oscars = oscars[["category", "name", "film", "winner"]]
oscars.head(3)

,category,name,film,winner
0,ACTOR,Richard Barthelmess,The Noose,False
1,ACTOR,Emil Jannings,The Last Command,True
2,ACTRESS,Louise Dresser,A Ship Comes In,False


In [5]:
oscars_cleaned = oscars.loc[(oscars['winner'] == True)]
oscars_cleaned.head(3)

,category,name,film,winner
1,ACTOR,Emil Jannings,The Last Command,True
3,ACTRESS,Janet Gaynor,7th Heaven,True
6,ART DIRECTION,William Cameron Menzies,The Dove;,True


# Combining Movies And Awards

In [6]:
combined_df = pd.merge(movies, oscars_cleaned, how="right", on = "film")
combined_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,Unnamed: 15,category,name,winner
0,0.0,Little Bear,USA,Bertrand Tavernier,Drama,3272600.0,'Round Midnight,R,1/17/86,133.0,7.4,Dexter Gordon,4028.0,David Rayfiel,1986.0,NaN,MUSIC (Original Score),Herbie Hancock,True
1,20000000.0,Regency Enterprises,USA,Steve McQueen,Biography,56671993.0,12 Years a Slave,R,2/21/86,134.0,8.1,Chiwetel Ejiofor,503763.0,John Ridley,2013.0,NaN,ACTRESS IN A SUPPORTING ROLE,Lupita Nyong'o,True
2,20000000.0,Regency Enterprises,USA,Steve McQueen,Biography,56671993.0,12 Years a Slave,R,2/21/86,134.0,8.1,Chiwetel Ejiofor,503763.0,John Ridley,2013.0,NaN,BEST PICTURE,"Brad Pitt, Dede Gardner, Jeremy Kleiner, Steve...",True


# Chaning Header Names

In [7]:
combined_df = combined_df.rename(columns = {"category": "Oscar Winning Category", "name":"Oscar Winner", 'winner': 'winner'})
#combined_df.head(5)

In [8]:
#combined_df = combined_df.drop("winner", axis = 1)
#combined_df.head(5)

In [9]:
combined_df.to_csv('movies&oscars.csv')

# Counting Awards and Nominations


In [10]:
nomination = oscars['film'].value_counts()
nomination
nomination = pd.DataFrame(nomination)
nomination = nomination.reset_index()
nomination = nomination.rename(columns = {"index":"film", "film":"Oscar Nominations"})
nomination.head(3)

,film,Oscar Nominations
0,A Star Is Born,25
1,Mutiny on the Bounty,15
2,Moulin Rouge,15


In [11]:
count = combined_df['film'].value_counts()
counts = pd.DataFrame(count)
counts.reset_index()
counts = counts.reset_index()
counts = counts.rename(columns = {"index":"film", 'film': "Oscar Award Count"})
counts.head(3)

,film,Oscar Award Count
0,The Lord of the Rings: The Return of the King,11
1,Ben-Hur,11
2,Titanic,11


In [12]:
combo_df = pd.merge(movies, counts, how="left", on = "film")
combo_df.head(5)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,Unnamed: 15,Oscar Award Count
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,NaN
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN
3,40000000,Warner Bros.,USA,Brian Helgeland,Biography,95001343,42,PG-13,1995,128,7.5,Chadwick Boseman,71006,Brian Helgeland,2013,NaN,NaN
4,13000000,Dollface,USA,Mark Christopher,Drama,16574731,54,R,1999,93,5.8,Ryan Phillippe,28791,Mark Christopher,1998,NaN,NaN


In [13]:
noms_df = pd.merge(combo_df, nomination, how="left", on = "film")
noms_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,Unnamed: 15,Oscar Award Count,Oscar Nominations
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,NaN,1.0
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,NaN
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,NaN


In [14]:
noms_df = noms_df.drop_duplicates()

# Average Score per Category

In [25]:
actors = noms_df[['star', 'score']]
actors_df = pd.DataFrame(actors)
actors_df.head(3)

,star,score
0,Elijah Wood,7.1
1,Jim Sturgess,6.8
2,Malcolm McDowell,5.0


In [26]:
actor_avg_rating = actors.groupby('star').mean()
actor_avg_rating.head(3)

,score
star,
'Weird Al' Yankovic,7.0
50 Cent,5.1
A.J. Cook,6.2


In [27]:
director = noms_df[['director', 'score']]
director_df = pd.DataFrame(director)
director_df.head(3)

,director,score
0,Shane Acker,7.1
1,Robert Luketic,6.8
2,Rob Zombie,5.0


In [28]:
director_avg_rating = director_df.groupby('director').mean()
director_avg_rating.head(3)

,score
director,
A.R. Murugadoss,7.3
Aamir Khan,8.5
Aaron Blaise,6.8


In [29]:
writer = noms_df[['writer', 'score']]
writer_df = pd.DataFrame(writer)
writer_df.head(3)

,writer,score
0,Pamela Pettler,7.1
1,Peter Steinfeld,6.8
2,Rob Zombie,5.0


In [30]:
writer_avg_rating = writer_df.groupby('writer').mean()
writer_avg_rating.head(3)

,score
writer,
'Weird Al' Yankovic,7.0
A.A. Milne,6.3
A.E. Hotchner,7.6


In [77]:
genre = noms_df[['genre', 'score']]
genre_df = pd.DataFrame(genre)
genre_df.head(3)

,genre,score
0,Animation,7.1
1,Crime,6.8
2,Horror,5.0


In [78]:
genre_avg_rating = genre_df.groupby('genre').mean()
genre_avg_rating

,score
genre,
Action,6.106086
Adventure,6.354847
Animation,6.746931
Biography,7.040390
Comedy,6.165769
Crime,6.756322
Drama,6.715720
Family,5.792857
Fantasy,5.775000


In [32]:
company = noms_df[['company', 'score']]
company_df = pd.DataFrame(company)
company_df.head(3)

,company,score
0,Focus Features,7.1
1,Columbia Pictures,6.8
2,Bow and Arrow Entertainment,5.0


In [82]:
company_avg_rating = company_df.groupby('company').mean()
company_avg_rating.head(3)

,score
company,
"""DIA"" Productions GmbH & Co. KG",6.1
1+2 Seisaku Iinkai,8.1
101st Street Films,2.3


In [36]:
rating = noms_df[['rating', 'score']]
rating_df = pd.DataFrame(rating)
rating_df.head(3)

,rating,score
0,PG-13,7.1
1,PG-13,6.8
2,R,5.0


In [38]:
rating_avg_rating = rating_df.groupby('rating').mean()
rating_avg_rating.head(3)

,score
rating,
B,6.900000
B15,6.300000
G,6.438095


In [39]:
country = noms_df[['country', 'score']]
country_df = pd.DataFrame(country)
country_df.head(3)

,country,score
0,USA,7.1
1,USA,6.8
2,UK,5.0


In [80]:
country_avg_rating = country_df.groupby('country').mean()
country_avg_rating.head(3)

,score
country,
Argentina,7.338462
Aruba,4.800000
Australia,6.685366


In [44]:
cleaned_df = pd.merge(noms_df, actor_avg_rating, how="left", on = "star")
cleaned_df.head(5)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score_x,star,votes,writer,year,Unnamed: 15,Oscar Award Count,Oscar Nominations,score_y
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,NaN,1.0,6.864286
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,NaN,6.875000
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,NaN,5.900000
3,40000000,Warner Bros.,USA,Brian Helgeland,Biography,95001343,42,PG-13,1995,128,7.5,Chadwick Boseman,71006,Brian Helgeland,2013,NaN,NaN,NaN,7.500000
4,13000000,Dollface,USA,Mark Christopher,Drama,16574731,54,R,1999,93,5.8,Ryan Phillippe,28791,Mark Christopher,1998,NaN,NaN,NaN,6.440000


In [45]:
cleaned_df = cleaned_df.rename(columns ={'score_x': 'score', 'score_y': 'star avg score'})
cleaned_df.head(5)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,Unnamed: 15,Oscar Award Count,Oscar Nominations,star avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,NaN,1.0,6.864286
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,NaN,6.875000
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,NaN,5.900000
3,40000000,Warner Bros.,USA,Brian Helgeland,Biography,95001343,42,PG-13,1995,128,7.5,Chadwick Boseman,71006,Brian Helgeland,2013,NaN,NaN,NaN,7.500000
4,13000000,Dollface,USA,Mark Christopher,Drama,16574731,54,R,1999,93,5.8,Ryan Phillippe,28791,Mark Christopher,1998,NaN,NaN,NaN,6.440000


In [46]:
cleaned_df = cleaned_df.drop("Unnamed: 15", axis = 1)
cleaned_df1 = cleaned_df.drop_duplicates()
cleaned_df1.head()

,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,1.0,6.864286
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,6.875000
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,5.900000
3,40000000,Warner Bros.,USA,Brian Helgeland,Biography,95001343,42,PG-13,1995,128,7.5,Chadwick Boseman,71006,Brian Helgeland,2013,NaN,NaN,7.500000
4,13000000,Dollface,USA,Mark Christopher,Drama,16574731,54,R,1999,93,5.8,Ryan Phillippe,28791,Mark Christopher,1998,NaN,NaN,6.440000


In [47]:
cleanest = pd.merge(cleaned_df1, director_avg_rating,how="left", on = "director" )
cleanest = cleanest.rename(columns ={'score_x': 'score', 'score_y': 'director avg score'})
cleanest.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,5.900000,5.666667


In [48]:
cleanest_df = pd.merge(cleanest, writer_avg_rating, how="left", on = "writer" )
cleanest_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score_x,star,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,score_y
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667


In [49]:
cleanest_df = cleanest_df.rename(columns ={'score_x': 'score', 'score_y': 'writer avg score'})
cleanest_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667


In [50]:
cleaner_df = pd.merge(cleanest_df, genre_avg_rating, how="left", on = "genre" )
cleaner_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,star,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,score_y
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,Elijah Wood,117268,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004


In [51]:
cleaner_df = cleaner_df.rename(columns ={'score_x': 'score', 'score_y': 'genre avg score'})
cleaner_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,star,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,Elijah Wood,117268,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004


In [52]:
cleaner1_df = pd.merge(cleaner_df, rating_avg_rating, how="left", on = "rating" )
cleaner1_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,score_y
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,117268,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931,6.254035
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,201705,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322,6.254035
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,11945,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004,6.453213


In [53]:
cleaner1_df = cleaner1_df.rename(columns ={'score_x': 'score', 'score_y': 'rating avg score'})
cleaner1_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,votes,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,rating avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,117268,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931,6.254035
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,201705,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322,6.254035
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,11945,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004,6.453213


In [54]:
cleaner2_df = pd.merge(cleaner1_df, country_avg_rating, how="left", on = "country" )
cleaner2_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,rating avg score,score_y
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931,6.254035,6.216790
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322,6.254035,6.216790
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004,6.453213,6.660602


In [55]:
cleaner2_df = cleaner2_df.rename(columns ={'score_x': 'score', 'score_y': 'country avg score'})
cleaner2_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,writer,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,rating avg score,country avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,Pamela Pettler,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931,6.254035,6.216790
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,Peter Steinfeld,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322,6.254035,6.216790
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,Rob Zombie,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004,6.453213,6.660602


In [56]:
cleaner3_df = pd.merge(cleaner2_df, company_avg_rating, how="left", on = "company" )
cleaner3_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,rating avg score,country avg score,score_y
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931,6.254035,6.216790,6.839474
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322,6.254035,6.216790,6.315741
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004,6.453213,6.660602,5.000000


In [57]:
cleaner3_df = cleaner3_df.rename(columns ={'score_x': 'score', 'score_y': 'company avg score'})
cleaner3_df.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,year,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,rating avg score,country avg score,company avg score
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,2009,NaN,1.0,6.864286,7.100000,7.100000,6.746931,6.254035,6.216790,6.839474
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,2008,NaN,NaN,6.875000,5.957143,6.200000,6.756322,6.254035,6.216790,6.315741
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,2016,NaN,NaN,5.900000,5.666667,5.666667,5.687004,6.453213,6.660602,5.000000


In [58]:
cleaner3_df.to_csv('updated.csv')

# Adding Golden globes data

In [59]:
csv_path = "csv/movies_cleaned.csv"
films = pd.read_csv(csv_path, encoding = "ISO-8859-1")
films.head(3)

,Unnamed: 0,budget,company,country,director,genre,gross,film,rating,released,runtime,score,star,votes,writer,year,ceremony,category,nominee,win
0,0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,9/9/09,79,7.1,Elijah Wood,117268,Pamela Pettler,2009,NaN,NaN,NaN,NaN
1,1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,3/28/08,123,6.8,Jim Sturgess,201705,Peter Steinfeld,2008,NaN,NaN,NaN,NaN
2,2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,9/16/16,102,5.0,Malcolm McDowell,11945,Rob Zombie,2016,NaN,NaN,NaN,NaN


In [60]:
films = films.drop("Unnamed: 0", axis = 1)
films_cleaned = films.drop_duplicates()


In [61]:
films_cleaned.to_csv('movies_cleaned.csv')

In [62]:
winners = films_cleaned.loc[films['win']==True]
wins = winners['film'].value_counts()
wins = pd.DataFrame(wins)
wins.reset_index()
wins = wins.reset_index()
wins = wins.rename(columns = {"index":"film","film": "Golden Globes Awards"})
wins

,film,Golden Globes Awards
0,Alice,7
1,La La Land,6
2,Arthur,3
3,Working Girl,3
4,The Truman Show,3
...,...,...
197,Mrs. Doubtfire,1
198,The Alamo,1
199,The English Patient,1
200,Buster,1


In [68]:
awards = pd.merge(cleaner3_df, wins, how='left', on = 'film')
awards.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,Oscar Award Count,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,rating avg score,country avg score,company avg score,Golden Globes Awards
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,NaN,1.0,6.864286,7.100000,7.100000,6.746931,6.254035,6.216790,6.839474,NaN
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,NaN,NaN,6.875000,5.957143,6.200000,6.756322,6.254035,6.216790,6.315741,NaN
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,NaN,NaN,5.900000,5.666667,5.666667,5.687004,6.453213,6.660602,5.000000,NaN


In [69]:
nm = films_cleaned.loc[films_cleaned['win'] != 'NaN']
nm = nm['film'].value_counts()
nm = pd.DataFrame(nm)
nm.reset_index()
nm = nm.reset_index()
nm = nm.rename(columns = {"index":"film","film": "Golden Globes Nominations"})
nm

,film,Golden Globes Nominations
0,Taxi '98,20
1,Taxi '04,20
2,Alice,13
3,Fargo,11
4,Weeds,8
...,...,...
6815,In a World...,1
6816,50 First Dates,1
6817,Super Troopers,1
6818,DrÃÂÃÂ¡cula: Muerto pero feliz,1


# Merging Golden Globe data and cleaning up DataFrame

In [70]:
gg = pd.merge(awards, nm, how='left', on = 'film')
gg.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,Oscar Nominations,star avg score,director avg score,writer avg score,genre avg score,rating avg score,country avg score,company avg score,Golden Globes Awards,Golden Globes Nominations
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,1.0,6.864286,7.100000,7.100000,6.746931,6.254035,6.216790,6.839474,NaN,1.0
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,NaN,6.875000,5.957143,6.200000,6.756322,6.254035,6.216790,6.315741,NaN,1.0
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,NaN,5.900000,5.666667,5.666667,5.687004,6.453213,6.660602,5.000000,NaN,1.0


In [71]:
gg['Oscar Award Count'] = gg["Oscar Award Count"].fillna(0)
gg['Oscar Nominations'] = gg["Oscar Nominations"].fillna(0)
gg['Golden Globes Awards'] = gg["Golden Globes Awards"].fillna(0)
gg['Golden Globes Nominations'] = gg["Golden Globes Nominations"].fillna(0)

In [72]:
gg['Total Nominations']= (gg['Oscar Nominations'] + gg['Golden Globes Nominations'])
gg['Total Awards'] = (gg['Oscar Award Count'] + gg['Golden Globes Awards'])
gg.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,director avg score,writer avg score,genre avg score,rating avg score,country avg score,company avg score,Golden Globes Awards,Golden Globes Nominations,Total Nominations,Total Awards
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,7.100000,7.100000,6.746931,6.254035,6.216790,6.839474,0.0,1.0,2.0,0.0
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,5.957143,6.200000,6.756322,6.254035,6.216790,6.315741,0.0,1.0,1.0,0.0
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,5.666667,5.666667,5.687004,6.453213,6.660602,5.000000,0.0,1.0,1.0,0.0


In [73]:
gg = gg.rename(columns = {'score': 'movie score'})
gg.head(3)

,budget,company,country,director,genre,gross,film,rating,released,runtime,...,director avg score,writer avg score,genre avg score,rating avg score,country avg score,company avg score,Golden Globes Awards,Golden Globes Nominations,Total Nominations,Total Awards
0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,79,...,7.100000,7.100000,6.746931,6.254035,6.216790,6.839474,0.0,1.0,2.0,0.0
1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,123,...,5.957143,6.200000,6.756322,6.254035,6.216790,6.315741,0.0,1.0,1.0,0.0
2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,102,...,5.666667,5.666667,5.687004,6.453213,6.660602,5.000000,0.0,1.0,1.0,0.0


In [74]:
gg.to_csv('final.csv')


In [1]:
queen = gg.loc[gg['film'] == "Batman Returns"]
queen

NameError: name 'gg' is not defined